# Input data requirements

The input ice sheet model should be a netCDF file. 


### `Lithk` variable
The uploaded model to contain thickness data (the `lithk` variable) for the comparison.


In [1]:
import os,sys


# Get the current working directory
current_directory = os.getcwd()
# Navigate two levels up to reach main dir
parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir, os.pardir))
# Insert the specific directory into the system path
sys.path.insert(0, os.path.join(parent_directory, 'bin', 'IMBIE'))
from imbie_utils import *


### Configure IMBIE comparison

In [2]:
# Define the flag for the ice sheet region Greenland or Antarctica
icesheet = "GIS"# Change to "AIS" or "GIS"

# Set start and end dates
start_date = '2006-01-01'
end_date ='2014-01-01'

#Set density of ice
rho_ice = 918 # (kg/m^3)

#output file dirctory
output_path='/home/jovyan/CmCt/notebooks/IMBIE/'

# Set model model path,shapefile path and projection and IMBIE csv_file
if icesheet == "GIS":
    projection = "EPSG:3413"  # Greenland
    #Set the model data dir path
    # nc_filename='/home/jovyan/shared-public/CmCt/models/ISMIP6/lithk_GIS_JPL_ISSM_asmb.nc'
    nc_filename='/home/jovyan/shared-public/CmCt/models/ISMIP6/lithk_GIS_ILTS_PIK_SICOPOLIS1_historical.nc'
    #Set the shape data dir path
    shape_filename = "/home/jovyan/CmCt/data/IMBIE/Greenland_Basins_PS_v1.4.2/Greenland_Basins_PS_v1.4.2.shp"
    #Set the observation data dir path
    # obs_filename = '/home/jovyan/CmCt/data/IMBIE/imbie_greenland_2022_Gt_partitioned_v0.csv'
    obs_filename = '/home/jovyan/CmCt/data/IMBIE/imbie_greenland_2021_Gt.csv'

    ##Set the Region observation data dir path
    obs_east_filename = None
    obs_west_filename = None
    obs_peninsula_filename = None
    
    
elif icesheet== "AIS":
    projection = "EPSG:3031"  # Antarctica    
    #Set the model data dir path
    nc_filename='/home/jovyan/CmCt/notebooks/Gravimetry/lithk_AIS_AWI_PISM1_hist_std.nc'
    #Set the shape data dir path
    shape_filename = "/home/jovyan/CmCt/data/IMBIE/ANT_Basins_IMBIE2_v1.6/ANT_Basins_IMBIE2_v1.6.shp"
    #Set the observation data dir path
    obs_filename = '/home/jovyan/CmCt/data/IMBIE/imbie_antarctica_2022_Gt_partitioned_v0.csv'
    # obs_filename = '/home/jovyan/CmCt/notebooks/IMBIE/imbie_antarctica_2021_Gt.csv'
    
    ##Set the Region observation data dir path
    obs_east_filename = '/home/jovyan/CmCt/data/IMBIE/imbie_east_antarctica_2022_Gt_partitioned_v0.csv'
    obs_west_filename = '/home/jovyan/CmCt/data/IMBIE/imbie_west_antarctica_2022_Gt_partitioned_v0.csv'
    obs_peninsula_filename= '/home/jovyan/CmCt/data/IMBIE/imbie_antarctic_peninsula_2022_Gt_partitioned_v0.csv'

else:
    raise ValueError("Invalid iceshee value. Must be 'Greenland' or 'Antarctica'.")


# Select  variable for mass balance comparision
mass_balance_column="Cumulative mass balance (Gt)"# "Cumulative dynamics mass balance anomaly (Gt)"
if mass_balance_column == "Cumulative mass balance (Gt)":
    mass_balance_type = "total"
elif mass_balance_column == "Cumulative dynamics mass balance anomaly (Gt)":
    mass_balance_type = "dynamic"

In [3]:
# Check if  observation file exists
if not os.path.exists(obs_filename):
    raise FileNotFoundError(f"Observation file not found: {obs_filename}")

# Check if model file exists    
if not os.path.exists(nc_filename):
    raise FileNotFoundError(f"Model file not found: {nc_filename}")


if icesheet== "AIS":   
    # Check if regional observation files exist 
    if not os.path.exists(obs_east_filename):
        raise FileNotFoundError(f"Observation file not found: {obs_east_filename}")
    if not os.path.exists(obs_west_filename):
        raise FileNotFoundError(f"Observation file not found: {obs_west_filename}")
    if not os.path.exists(obs_peninsula_filename):
        raise FileNotFoundError(f"Observation file not found: {obs_peninsula_filename}")

## Load the model data

In [4]:
#Model data

nc_filename='/home/jovyan/shared-public/CmCt/models/ISMIP6/lithk_GIS_BGC_BISICLES_historical.nc'
gis_ds = xr.open_dataset(nc_filename)
lithk = gis_ds['lithk']
time_var = gis_ds['time']

## Mass change comparision processing

In [5]:
# Process the model data and get mass balance and basin sums
basin_result = process_model_data(nc_filename,start_date, end_date,rho_ice,projection,shape_filename,icesheet)
            

# Process the IMBIE data and get results in a dictionary
results = process_IMBIE(obs_filename, start_date, end_date, icesheet,  basin_result,mass_balance_column,obs_east_filename, obs_west_filename, 
                        obs_peninsula_filename)


The selected dates 2006-01-01 and 2014-01-01 are within the range of the model data.


## Display result and write it to csv file

In [15]:
# Extract the base name of the nc file (without .nc extension)
nc_base_filename = os.path.basename(nc_filename).replace('.nc', '')

# Create the CSV filename by combining the output path and the base nc filename with .csv extension
csv_filename = os.path.join(output_path, f"{nc_base_filename}.csv")

write_and_display_mass_change_comparison(icesheet, basin_result, results,mass_balance_type,start_date,end_date,csv_filename)
  



Mass change comparison (total): 2006-01-01 - 2014-01-01
Basin      Model mass change (Gt)    IMBIE mass change (Gt)    Residual (Gt)            
CE         14.27                     --                        --                       
CW         -175.61                   --                        --                       
NE         2.26                      --                        --                       
NO         -78.80                    --                        --                       
NW         -160.12                   --                        --                       
SE         -70.65                    --                        --                       
SW         -296.07                   --                        --                       
Total      -764.72                   -2170.68                  -1405.96                 
Writing data to CSV file: /home/jovyan/CmCt/notebooks/IMBIE/lithk_GIS_BGC_BISICLES_historical.csv
